<center><h1> 42380 Supply Chain Analytics</h1></center>

<h2 align="center">Assignment 1: Traffic flow fundamentals</h2></br>

<center>Adamopoulos Thomas Student no. : s213163</center>
<center>Zerzis Dimitris Student no. : s210911</center>
<center>Apeni Eben Student no. : s216084</center>
<center>Guitart Gendre Andreu Student no. : s213260</center>
<br>
<center>All members contributed</center>
<center>Code predominantly Dimitris and Thomas</center>
<center>Questions predominantly Eben and Andreu</center>

# Part I: Demand analysis

### 1. Determine the demand distribution (mean and standard deviation) for salmon at each DC on each of the days in the planning horizon.  <br>
### 2. What is the benefit of having salmon in each of the three products? Would there have been downsides to having three different kinds of fish in the three different products, even if they all cost the same per kilogram?

1. Completed in attatched Excel <br>
<br>
2. The primary benefit of having only salmon in each product simplifies the job of the supplier as there are 3 variations. Had there been 3 different types of fish there would be 9 variations. Consequently, the demand for all three different products can be treated as one. Simplifying both, the model for the inventory and for the route generation.<br><br> Regarding the inventory, this would have to supply all the demand and not take into account to cover the demand that each product has. It would be only one distribution. Also, It is important to bear in mind that calculations of the costs will be easier as well. The uncertainty would be covered more easily and the probability to suffer expenses due to the perishable goods will be reduced.<br><br>To give an example, if we had different types of fish for each product we would need the demand of each fish product to find the inventory for each DC. Therefore there would be more variables and the calculation would be slightly more complex. Resulting in increased cost. In addition to the storage of the different fish as each one would be governed by different timescales.<br><br>For the route generation, it is also easier. There will be fewer partitions and the vehicles will have more freedom to do their routes. This means that would be a combinatorial optimization in the other case the problem would be more complex and is it known that the number of possible combinations will grow exponentially.


---------------------------------------------------------------------------------------------------------------------------------------

# Part II: Solving the problem

## <b>Q1.</b> Build and solve an inventory routing model in Julia in order to determine the and the optimal inventory levels for the DCs, assuming there is no uncertainty (so only use salmon demand distribution). Report on the routes used and the inventory levels over the different days.
Hint: If your model is correct, you should get an optimal cost of 4481.57.

In [3]:
# Mandatory imports 

using JuMP
using Gurobi
using Random
using Distributions

In [7]:
#------
# DATA
#------

# holding cost per unit h = 0.025 
K = 3;    # number vehicles

#I = 38    # Number of customers                    (39 number of nodes, node 1 is the depot)
N = 6     # Number of DCs +supplier
T = 8;    # Number of periods


# Holding cost per unit per DC
h = 0.025  

# Capacity of trucks in kg
D = [900 900 900]

# Storage Capacity per DC
C = [2500 502 488 220 486 742]  # Storage Capacity per DC +supplier (2500>2438 sum of DCs)

# Initial inventory per DC
I_0 = [0 98.8 153.9 42.4 23.4 85.7] # Initial inventory per DC

#Mean demand dc/day (10 planning horizons)
d=[0 0 0 0 0 0 0 0;
75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9;
62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9;
67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2;
102.3 156 156 156 156 119.05 65.35 102.3;
107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4]

# Standard deviation of Demand
std=[
0 0 0 0 0 0 0 0;
11.87 11.87 11.87 11.87 11.87 14.69 14.69 11.87;
10.91 13.89 13.89 13.89 13.89 14.14 11.22 10.91;
11.27 11.27 11.27 11.27 11.27 12.86 12.86 11.27;
13.19 16.03 16.03 16.03 16.03 15.05 11.98 13.19;
13.04 14.00 14.00 14.00 14.00 11.08 9.84 13.04]


#Transportation cost per unit
c=[0 140 434 389 419 125;
140 0 300 455 400 97;
434 300 0 609 417 330;
389 455 609 0 256 358;
419 400 417 256 0 316;
125 97 330 358 316 0]

print("")

In [8]:
#------
# MODEL
#------

model = Model(Gurobi.Optimizer);

@variable(model, I[1:N,0:T] >= 0)          # Inventory level per period
@variable(model, q[1:N,1:K,1:T] >= 0)      # Quantity delivered to customer i by vehicle k at start of period T 
@variable(model, x[1:N,1:N,1:K,1:T], Bin); # Truck k leaves DCi and goes TO DCj
@variable(model, y[1:N,1:K,1:T], Bin);     # Truck k arrives at DCi
@variable(model, z[1:N,1:K,1:T] >= 0);     # Load of truck k when arriving at node i

@objective(model, Min, sum(sum(h*I[i,t]  for i in 2:N) for t in 1:T) #check
                      +sum(sum(sum(sum( c[i,j]*x[i,j,k,t] for i in 1:N) for j in 1:N) for k = 1:K) for t in 1:T)); 



# Multi - period VRP model constrains
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(i == v ? 0 : x[i,v,k,t] for i = 1:N) == y[v,k,t]); #!
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(j == v ? 0 : x[v,j,k,t] for j = 1:N) == y[v,k,t]); #!

# Subtour elimination constrains    
@constraint(model,[i = 2:N, j = 2:N, k = 1:K,t = 1:T], z[i,k,t] - d[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d)); # Load Constrains logic (i=j=2 substrack depot)

#Inventory Constrains #!!
@constraint(model, [i = 2:N], I[i,0] == I_0[i])    
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) == I[i,t] + d[i,t]); 
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) <= C[i]); 

# Inventory & VRP constrains
@constraint(model,[i=2:N,k=1:K,t=1:T], q[i,k,t] <= C[i]*y[i,k,t]); # Constraint which ensures y is 1 when they receive a quantity    
@constraint(model,[k = 1:K,t = 1:T],  sum(q[i,k,t] for i = 2:N) <= D[k]*y[1,k,t]);# Avoid subtours  


Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-30


In [9]:
#------
# SOLVE
#------

optimize!(model)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0x9c1140df
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+03]
Found heuristic solution: objective 10803.506250
Presolve removed 160 rows and 213 columns
Presolve time: 0.00s
Presolved: 957 rows, 1137 columns, 3764 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 2.649403e+03, 819 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2649.40295    0   88 10803.5063 2649.40295  75.5%     -    0s
H    0     0     

In [11]:
for t = 1:T
    println("-----")
    println("Day", t)

    for k = 1:K
        has_route = false;
        println("Truck ", k, ": ")
        
        for i = 1:N
            for j = 1:N
                if (value(x[i,j,k,t]) >= 0.9999)
                    if i != j
                        print(" ",i,"->",j," ")
                        has_route = true;
                    end
                end
                
            end
        end
        print("")
        if has_route
            print("");
        else
            print("--NO ROUTE--")
        end
        println("")
    end    
end

-----
Day1
Truck 1: 
 1->6  6->1 
Truck 2: 
--NO ROUTE--
Truck 3: 
 1->2  2->3  3->5  4->1  5->4 
-----
Day2
Truck 1: 
--NO ROUTE--
Truck 2: 
--NO ROUTE--
Truck 3: 
--NO ROUTE--
-----
Day3
Truck 1: 
--NO ROUTE--
Truck 2: 
 1->4  4->5  5->6  6->1 
Truck 3: 
--NO ROUTE--
-----
Day4
Truck 1: 
--NO ROUTE--
Truck 2: 
--NO ROUTE--
Truck 3: 
--NO ROUTE--
-----
Day5
Truck 1: 
--NO ROUTE--
Truck 2: 
--NO ROUTE--
Truck 3: 
--NO ROUTE--
-----
Day6
Truck 1: 
 1->4  2->1  3->2  4->5  5->3 
Truck 2: 
--NO ROUTE--
Truck 3: 
--NO ROUTE--
-----
Day7
Truck 1: 
--NO ROUTE--
Truck 2: 
--NO ROUTE--
Truck 3: 
--NO ROUTE--
-----
Day8
Truck 1: 
--NO ROUTE--
Truck 2: 
--NO ROUTE--
Truck 3: 
--NO ROUTE--


In [12]:
# Printing the optimal solution 

if termination_status(model) == MOI.OPTIMAL
   println("Optimal solution:")
   println("Objective function = $(objective_value(model))")
else
   println("No solution found")
end

Optimal solution:
Objective function = 4481.57


## <b>Q2.</b> Build a simulation in Julia to estimate if the fill rate target can be met. Report on the fill rates for the different DCs over the different days. Below is some code to help you get started. The situation is simulated 10000 times and each time actual demands are generated using the demand distribution for salmon (given by the matrices d and std). The amount delivered to each DC on each day is obtained from the solution of the model from (1)above.

Hint: If your simulation works correctly (and you use the same random seed and number of iterations as
above), the probability of meeting 95% of demand for DC 5 on day 8 should be 0.5551 (thus not meeting the
target).


In [13]:
# Finding the fill ratio

for i=2:N
    println("For the DC= ",i )
    for t=1:T 
        println("On day ",t," the fill rate was:",(max(value(d[i,t])+min(value(I[i,t]),0),0))/value(d[i,t]))
    end
end

For the DC= 2
On day 1 the fill rate was:1.0
On day 2 the fill rate was:1.0
On day 3 the fill rate was:1.0
On day 4 the fill rate was:1.0
On day 5 the fill rate was:1.0
On day 6 the fill rate was:1.0
On day 7 the fill rate was:1.0
On day 8 the fill rate was:1.0
For the DC= 3
On day 1 the fill rate was:1.0
On day 2 the fill rate was:1.0
On day 3 the fill rate was:1.0
On day 4 the fill rate was:1.0
On day 5 the fill rate was:1.0
On day 6 the fill rate was:1.0
On day 7 the fill rate was:1.0
On day 8 the fill rate was:1.0
For the DC= 4
On day 1 the fill rate was:1.0
On day 2 the fill rate was:1.0
On day 3 the fill rate was:1.0
On day 4 the fill rate was:1.0
On day 5 the fill rate was:1.0
On day 6 the fill rate was:1.0
On day 7 the fill rate was:1.0
On day 8 the fill rate was:1.0
For the DC= 5
On day 1 the fill rate was:1.0
On day 2 the fill rate was:1.0
On day 3 the fill rate was:1.0
On day 4 the fill rate was:1.0
On day 5 the fill rate was:1.0
On day 6 the fill rate was:1.0
On day 7 the f

In [14]:
# Random.seed!(0);
Fullness=zeros(6,8)
for r = 1:10000
    actual_demand = d + randn(6,8).*std;
    for i=1:N
        I_1=I_0[i];
        for t=1:T
            #fillrate=0;
            delivered=0;
            for k=1:K
                delivered = delivered + value(q[i,k,t]);
            end
            I_1=I_1+delivered-value(actual_demand[i,t]);
            fillrate=(max(value(actual_demand[i,t])+min(I_1,0),0))/value(actual_demand[i,t]);
            if fillrate>=0.95
                Fullness[i,t]=Fullness[i,t] + 1
            end
        end
    end
end

In [15]:
print("The fill ratio of each DC per day can be shonwn in the matrix below:\n")
Fullness/10000

The fill ratio of each DC per day can be shonwn in the matrix below:


6×8 Matrix{Float64}:
 0.0  0.0     0.0  0.0     0.0     0.0     0.0     0.0
 1.0  1.0     1.0  0.9999  0.5563  1.0     0.9916  0.5405
 1.0  1.0     1.0  1.0     0.5608  0.9991  0.9676  0.5245
 1.0  0.5911  1.0  0.9994  0.5526  1.0     0.9892  0.5365
 1.0  0.6526  1.0  1.0     0.6282  1.0     0.9957  0.5419
 1.0  1.0     1.0  1.0     1.0     1.0     0.9996  0.5557

## <b>Q3.</b> In case both storage and truck capacity is inflated 10 times, you should notice from your analysis above that the fill rate target can be met using a much smaller safety factor. Use what you have learned in this course to explain why this is the case.<br>
<br>Hint: Look at the difference in the optimal routes used in the solution with the given capacities compared to
the solution when capacities are inflated.


In [16]:
a=0.98
za=quantile.(Normal(0,1),a)
d_1 = d + za.*std;

In [17]:
#------
# MODEL
#------

model = Model(Gurobi.Optimizer);

@variable(model, I[1:N,0:T] >= 0)          # Inventory level per period
@variable(model, q[1:N,1:K,1:T] >= 0)      # Quantity delivered to customer i by vehicle k at start of period T 
@variable(model, x[1:N,1:N,1:K,1:T], Bin); # Truck k leaves DCi and goes TO DCj
@variable(model, y[1:N,1:K,1:T], Bin);     # Truck k arrives at DCi
@variable(model, z[1:N,1:K,1:T] >= 0);     # Load of truck k when arriving at node i

@objective(model, Min, sum(sum(h*I[i,t]  for i in 2:N) for t in 1:T) #check
                      +sum(sum(sum(sum( c[i,j]*x[i,j,k,t] for i in 1:N) for j in 1:N) for k = 1:K) for t in 1:T)); 



# Multi - period VRP model constrains
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(i == v ? 0 : x[i,v,k,t] for i = 1:N) == y[v,k,t]); #!
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(j == v ? 0 : x[v,j,k,t] for j = 1:N) == y[v,k,t]); #!

# Subtour elimination constrains    
@constraint(model,[i = 2:N, j = 2:N, k = 1:K,t = 1:T], z[i,k,t] - d_1[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d_1)); # Load Constrains logic (i=j=2 substrack depot)

#Inventory Constrains #!!
@constraint(model, [i = 2:N], I[i,0] == I_0[i])    
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) == I[i,t] + d_1[i,t]); 
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) <= C[i]); 

# Inventory & VRP constrains
@constraint(model,[i=2:N,k=1:K,t=1:T], q[i,k,t] <= C[i]*y[i,k,t]); # Constraint which ensures y is 1 when they receive a quantity    
@constraint(model,[k = 1:K,t = 1:T],  sum(q[i,k,t] for i = 2:N) <= D[k]*y[1,k,t]);  # Avoid subtours

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-30


In [18]:
#------
# SOLVE
#------

optimize!(model)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0x9353f38d
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+03]
Presolve removed 160 rows and 212 columns
Presolve time: 0.01s
Presolved: 957 rows, 1138 columns, 3763 nonzeros
Variable types: 274 continuous, 864 integer (864 binary)

Root relaxation: objective 3.669626e+03, 646 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3669.62580    0   93          - 3669.62580      -     -    0s
H    0     0                    12136.763039 3669.62580  69.8%    

In [19]:
# Printing the optimal solution 

if termination_status(model) == MOI.OPTIMAL
   println("Optimal solution:")
   println("Objective function = $(objective_value(model))")
else
   println("No solution found")
end

Optimal solution:
Objective function = 5777.806675993788


In [20]:
# Random.seed!(0);
Fullness=zeros(6,8)
for r = 1:10000
    actual_demand = d + randn(6,8).*std;
    for i=1:N
        I_1=I_0[i];
        for t=1:T
            #fillrate=0;
            delivered=0;
            for k=1:K
                delivered = delivered + value(q[i,k,t]);
            end
            I_1=I_1+delivered-value(actual_demand[i,t]);
            fillrate=(max(value(actual_demand[i,t])+min(I_1,0),0))/value(actual_demand[i,t]);
            if fillrate>=0.95
                Fullness[i,t]=Fullness[i,t] + 1
            end
        end
    end
end

In [21]:
print("The fill ratio of each DC per day can be shonwn in the matrix below:\n")
Fullness/10000

The fill ratio of each DC per day can be shonwn in the matrix below:


6×8 Matrix{Float64}:
 0.0  0.0     0.0  0.0  0.0  0.0  0.0     0.0
 1.0  1.0     1.0  1.0  1.0  1.0  1.0     1.0
 1.0  1.0     1.0  1.0  1.0  1.0  0.9993  1.0
 1.0  0.9996  1.0  1.0  1.0  1.0  1.0     1.0
 1.0  1.0     1.0  1.0  1.0  1.0  1.0     1.0
 1.0  1.0     1.0  1.0  1.0  1.0  1.0     1.0

---------------------------------------------------------------------------------------------------------------------------------------

# Part III: Capacity analysis


## <b>Q1.</b> Analyze the capacity utilization of your solution from Part II. Calculate the % of storage capacity utilized at each DC on each day, and calculate the % of truck capacity utilized at the start of each trip. Based on this analysis, which capacity (storage or truck) seems to be the bottleneck?

In [22]:
#We created a mew matrix to calculate  the capacity of each dc and day 
create matrix for capacity of each dc to each day
one = ones(6,8);
capacity = transpose(C) .* one;

inventory = value.(I[6,8] + d_1[6,8]);
inventory ./ capacity .*100

LoadError: syntax: extra token "matrix" after end of expression

In [23]:
# % of each truck capacity utilized on each day, DC x day
truck_delivered = value.(sum(q[i,1:3,1:8] for i in 1:5))
truck_utility = truck_delivered/900 .*100

3×8 Matrix{Float64}:
   0.0  0.0   0.0    0.0  100.0  0.0  68.8219  0.0
  44.8  0.0  55.254  0.0    0.0  0.0   0.0     0.0
 100.0  0.0   0.0    0.0    0.0  0.0   0.0     0.0

## <b>Q2.</b> Consider three alternative capacity scenarios:

### (a) Truck capacity is 10 times more than in the current situation

In [24]:
D_1=10*D


1×3 Matrix{Int64}:
 9000  9000  9000

In [25]:
#------
# MODEL
#------

model = Model(Gurobi.Optimizer);

@variable(model, I[1:N,0:T] >= 0)          # Inventory level per period
@variable(model, q[1:N,1:K,1:T] >= 0)      # Quantity delivered to customer i by vehicle k at start of period T 
@variable(model, x[1:N,1:N,1:K,1:T], Bin); # Truck k leaves DCi and goes TO DCj
@variable(model, y[1:N,1:K,1:T], Bin);     # Truck k arrives at DCi
@variable(model, z[1:N,1:K,1:T] >= 0);     # Load of truck k when arriving at node i

@objective(model, Min, sum(sum(h*I[i,t]  for i in 2:N) for t in 1:T) #check
                      +sum(sum(sum(sum( c[i,j]*x[i,j,k,t] for i in 1:N) for j in 1:N) for k = 1:K) for t in 1:T)); 



# Multi - period VRP model constrains
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(i == v ? 0 : x[i,v,k,t] for i = 1:N) == y[v,k,t]); #!
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(j == v ? 0 : x[v,j,k,t] for j = 1:N) == y[v,k,t]); #!

# Subtour elimination constrains    
@constraint(model,[i = 2:N, j = 2:N, k = 1:K,t = 1:T], z[i,k,t] - d[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d)); # Load Constrains logic (i=j=2 substrack depot)

#Inventory Constrains #!!
@constraint(model, [i = 2:N], I[i,0] == I_0[i])    
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) == I[i,t] + d[i,t]); 
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) <= C[i]); 

# Inventory & VRP constrains
@constraint(model,[i=2:N,k=1:K,t=1:T], q[i,k,t] <= C[i]*y[i,k,t]); # Constraint which ensures y is 1 when they receive a quantity    
@constraint(model,[k = 1:K,t = 1:T],  sum(q[i,k,t] for i = 2:N) <= D_1[k]*y[1,k,t]);# Avoid subtours

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-30


In [26]:
#------
# SOLVE
#------

optimize!(model)


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0x3e6cae4c
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+03]
Found heuristic solution: objective 10822.523750
Presolve removed 160 rows and 213 columns
Presolve time: 0.01s
Presolved: 957 rows, 1137 columns, 3764 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 2.095202e+03, 695 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2095.20175    0   95 10822.5238 2095.20175  80.6%     -    0s
H    0     0     

In [27]:
# Printing the optimal solution 

if termination_status(model) == MOI.OPTIMAL
   println("Optimal solution:")
   println("Objective function = $(objective_value(model))")
else
   println("No solution found")
end

Optimal solution:
Objective function = 4288.975


In [28]:
print("The effect of A scenario on the total cost equals to ", 4481 - 4289)

The effect of A scenario on the total cost equals to 192

In [29]:
# Random.seed!(0);
Fullness=zeros(6,8)
for r = 1:10000
    actual_demand = d + randn(6,8).*std;
    for i=1:N
        I_1=I_0[i];
        for t=1:T
            #fillrate=0;
            delivered=0;
            for k=1:K
                delivered = delivered + value(q[i,k,t]);
            end
            I_1=I_1+delivered-value(actual_demand[i,t]);
            fillrate=(max(value(actual_demand[i,t])+min(I_1,0),0))/value(actual_demand[i,t]);
            if fillrate>=0.95
                Fullness[i,t]=Fullness[i,t] + 1
            end
        end
    end
end

In [30]:
print("The fill ratio of each DC per day can be shonwn in the matrix below:\n")
Fullness/10000

The fill ratio of each DC per day can be shonwn in the matrix below:


6×8 Matrix{Float64}:
 0.0  0.0  0.0     0.0     0.0     0.0     0.0     0.0
 1.0  1.0  1.0     0.9995  0.5555  1.0     0.9908  0.5445
 1.0  1.0  1.0     0.9999  0.5717  0.9992  0.9698  0.544
 1.0  1.0  0.5673  0.9995  0.5537  1.0     0.9905  0.5357
 1.0  1.0  0.6242  1.0     0.5883  1.0     0.996   0.5479
 1.0  1.0  0.5995  1.0     1.0     1.0     0.999   0.5538

### (b) Storage capacity is 10 times more than in the current situation

In [31]:
C_1 = C*10

1×6 Matrix{Int64}:
 25000  5020  4880  2200  4860  7420

In [32]:
#------
# MODEL
#------

model = Model(Gurobi.Optimizer);

@variable(model, I[1:N,0:T] >= 0)          # Inventory level per period
@variable(model, q[1:N,1:K,1:T] >= 0)      # Quantity delivered to customer i by vehicle k at start of period T 
@variable(model, x[1:N,1:N,1:K,1:T], Bin); # Truck k leaves DCi and goes TO DCj
@variable(model, y[1:N,1:K,1:T], Bin);     # Truck k arrives at DCi
@variable(model, z[1:N,1:K,1:T] >= 0);     # Load of truck k when arriving at node i

@objective(model, Min, sum(sum(h*I[i,t]  for i in 2:N) for t in 1:T) #check
                      +sum(sum(sum(sum( c[i,j]*x[i,j,k,t] for i in 1:N) for j in 1:N) for k = 1:K) for t in 1:T)); 



# Multi - period VRP model constrains
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(i == v ? 0 : x[i,v,k,t] for i = 1:N) == y[v,k,t]); #!
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(j == v ? 0 : x[v,j,k,t] for j = 1:N) == y[v,k,t]); #!

# Subtour elimination constrains    
@constraint(model,[i = 2:N, j = 2:N, k = 1:K,t = 1:T], z[i,k,t] - d[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d)); # Load Constrains logic (i=j=2 substrack depot)

#Inventory Constrains #!!
@constraint(model, [i = 2:N], I[i,0] == I_0[i])    
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) == I[i,t] + d[i,t]); 
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) <= C_1[i]); 

# Inventory & VRP constrains
@constraint(model,[i=2:N,k=1:K,t=1:T], q[i,k,t] <= C_1[i]*y[i,k,t]); # Constraint which ensures y is 1 when they receive a quantity    
@constraint(model,[k = 1:K,t = 1:T],  sum(q[i,k,t] for i = 2:N) <= D[k]*y[1,k,t]);# Avoid subtours

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-30


In [33]:
#------
# SOLVE
#------

optimize!(model)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0x35a09c8d
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 7e+03]
Found heuristic solution: objective 11689.402500
Presolve removed 162 rows and 213 columns
Presolve time: 0.01s
Presolved: 955 rows, 1137 columns, 3758 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 1.664604e+03, 638 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1664.60369    0   86 11689.4025 1664.60369  85.8%     -    0s
H    0     0     

In [34]:
# Printing the optimal solution 

if termination_status(model) == MOI.OPTIMAL
   println("Optimal solution:")
   println("Objective function = $(objective_value(model))")
else
   println("No solution found")
end

Optimal solution:
Objective function = 3311.369999999999


In [35]:
print("The effect of B scenario on the total cost equals to ", 4481 - 3311)

The effect of B scenario on the total cost equals to 1170

In [36]:
# Random.seed!(0);
Fullness=zeros(6,8)
for r = 1:10000
    actual_demand = d + randn(6,8).*std;
    for i=1:N
        I_1=I_0[i];
        for t=1:T
            #fillrate=0;
            delivered=0;
            for k=1:K
                delivered = delivered + value(q[i,k,t]);
            end
            I_1=I_1+delivered-value(actual_demand[i,t]);
            fillrate=(max(value(actual_demand[i,t])+min(I_1,0),0))/value(actual_demand[i,t]);
            if fillrate>=0.95
                Fullness[i,t]=Fullness[i,t] + 1
            end
        end
    end
end

In [37]:
print("The fill ratio of each DC per day can be shonwn in the matrix below:\n")
Fullness/10000

The fill ratio of each DC per day can be shonwn in the matrix below:


6×8 Matrix{Float64}:
 0.0     0.0  0.0     0.0  0.0  0.0     0.0     0.0
 0.9905  1.0  1.0     1.0  1.0  1.0     0.9915  0.5383
 1.0     1.0  1.0     1.0  1.0  0.9988  0.9702  0.5324
 1.0     1.0  1.0     1.0  1.0  1.0     0.9899  0.5517
 1.0     1.0  0.7823  1.0  1.0  1.0     0.997   0.555
 1.0     1.0  1.0     1.0  1.0  1.0     0.9991  0.5614

### (c) Both truck and storage capacity is 10 times more than in the current situation

In [38]:
#------
# MODEL
#------

model = Model(Gurobi.Optimizer);

@variable(model, I[1:N,0:T] >= 0)          # Inventory level per period
@variable(model, q[1:N,1:K,1:T] >= 0)      # Quantity delivered to customer i by vehicle k at start of period T 
@variable(model, x[1:N,1:N,1:K,1:T], Bin); # Truck k leaves DCi and goes TO DCj
@variable(model, y[1:N,1:K,1:T], Bin);     # Truck k arrives at DCi
@variable(model, z[1:N,1:K,1:T] >= 0);     # Load of truck k when arriving at node i

@objective(model, Min, sum(sum(h*I[i,t]  for i in 2:N) for t in 1:T) #check
                      +sum(sum(sum(sum( c[i,j]*x[i,j,k,t] for i in 1:N) for j in 1:N) for k = 1:K) for t in 1:T)); 



# Multi - period VRP model constrains
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(i == v ? 0 : x[i,v,k,t] for i = 1:N) == y[v,k,t]); #!
@constraint(model,[v = 1:N, k = 1:K,t = 1:T], sum(j == v ? 0 : x[v,j,k,t] for j = 1:N) == y[v,k,t]); #!

# Subtour elimination constrains    
@constraint(model,[i = 2:N, j = 2:N, k = 1:K,t = 1:T], z[i,k,t] - d_1[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d_1)); # Load Constrains logic (i=j=2 substrack depot)

#Inventory Constrains #!!
@constraint(model, [i = 2:N], I[i,0] == I_0[i])    
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) == I[i,t] + d_1[i,t]); 
@constraint(model, [i = 2:N,t = 1:T], I[i,t-1] + sum(q[i,k,t] for k = 1:K) <= C_1[i]); 

# Inventory & VRP constrains
@constraint(model,[i=2:N,k=1:K,t=1:T], q[i,k,t] <= C_1[i]*y[i,k,t]); # Constraint which ensures y is 1 when they receive a quantity    
@constraint(model,[k = 1:K,t = 1:T],  sum(q[i,k,t] for i = 2:N) <= D_1[k]*y[1,k,t]);# Avoid subtours

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-30


In [39]:
#------
# SOLVE
#------

optimize!(model)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0x25db5b6c
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 7e+03]
Found heuristic solution: objective 9715.5113311
Presolve removed 160 rows and 212 columns
Presolve time: 0.00s
Presolved: 957 rows, 1138 columns, 3763 nonzeros
Variable types: 274 continuous, 864 integer (864 binary)

Root relaxation: objective 3.773080e+02, 600 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  377.30803    0   93 9715.51133  377.30803  96.1%     -    0s
H    0     0     

In [40]:
# Printing the optimal solution 

if termination_status(model) == MOI.OPTIMAL
   println("Optimal solution:")
   println("Objective function = $(objective_value(model))")
else
   println("No solution found")
end

Optimal solution:
Objective function = 1975.2422511091265


In [41]:
print("The effect of C scenario : ", 4481 - 1975)

The effect of C scenario : 2506

In [42]:
# Random.seed!(0);
Fullness=zeros(6,8)
for r = 1:10000
    actual_demand = d + randn(6,8).*std;
    for i=1:N
        I_1=I_0[i];
        for t=1:T
            #fillrate=0;
            delivered=0;
            for k=1:K
                delivered = delivered + value(q[i,k,t]);
            end
            I_1=I_1+delivered-value(actual_demand[i,t]);
            fillrate=(max(value(actual_demand[i,t])+min(I_1,0),0))/value(actual_demand[i,t]);
            if fillrate>=0.95
                Fullness[i,t]=Fullness[i,t] + 1
            end
        end
    end
end

In [43]:
print("The fill ratio of each DC per day can be shonwn in the matrix below:\n")
Fullness/10000

The fill ratio of each DC per day can be shonwn in the matrix below:


6×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0
 1.0  1.0  1.0  1.0  1.0  1.0  0.9995  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0

## 3. In case both storage and truck capacity is inflated 10 times, you should notice from your analysis above that the fill rate target can be met using a much smaller safety factor. Use what you have learned in this course to explain why this is the case.
Hint: Look at the difference in the optimal routes used in the solution with the given capacities compared to the solution when capacities are inflated.

The fill rate ratio is effectively 100% for each DC on each day, as seen in the matrix therefore the current model is robust enough to fullfill all customers demands within the planning horizon, when there is an increase in both truck and storage capacity. As capacity increase uncerntainty decreases hence a smaller safety factor can still meet the fill rate target.

---------------------------------------------------------------------------------------------------------------------------------------

# Part IV: Flexibility analysis 

### Consider the situation where the customers provide their orders one day earlier than they currently do. Whatflexibility is there in your current solution (to Part II) for utilizing this information to reduce costs? In what waycan the inventory routing model be updated in order to create even more of this kind of flexibility? 

In this situation, if orders are received a day earlier than at present, the company will have more time to manage inventories and potentially update their forecasts accordingly giving more true reflection on the current and future demand. As vehicle routes are planned for the next 8 days this may allow time for adjustment in the vehicle routes/ inventory to reduce backlog. as a result reduce the costs. It goes without saying that the interest of any company is to be able to manage the inventory in the most economically viable way, otherwise, it could lead to unfulfilled orders. as these are perishable goods  all of which with unwelcomed results like the loss of customers. Therefore, what is trying to be said is that the interest is to stay updated, which means that receiving orders a day earlier can be beneficial to reducing costs. <br>

This could be due to the fact that, for example, the first way to reduce costs is with the safety factor which can be reduced as more information in advance will reduce the uncertainty and this will allow more accurate forecasting of the products allowing lower safety factor values less rate target and increasing the value of the warehouse space.
